In [1]:
import os
import requests
import csv
import argparse
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
import pandas as pd
import datetime, calendar


In [2]:
# Funció per fer el dataframe d'un mes
def __audiencia_del_mes(mes):
    """DOCSTRING_DE_FUNCION"""
    
    data = pd.DataFrame(columns=('Audiencia', 'Cadena', 'Fecha'))
    
    year = 2020
    month = mes

    num_days = calendar.monthrange(year, month)[1]
    days = [datetime.date(year, month, day) for day in range(1, num_days+1)]
    for day in days: 
        fecha = day.strftime('%Y-%m-%d')
        url = 'https://www.formulatv.com/audiencias/'+fecha
        req = requests.get(url)
        soup = BeautifulSoup(req.text, "lxml")

        # Busquem els ul
        ul = soup.h1.find_next_sibling("ul")
        cadenas1 = (" ".join([img['alt'] for img in ul.find_all('img', alt=True)]))
        cadenas1 = re.sub(re.compile('logo'),"",cadenas1)

        # Ho passem a llista
        cadenas1 = cadenas1.split(" ")
        # Treiem el percentatges, és a dir, el text de cada li
        # Escollim posar un espai com a separador

        audiencias1 = ul.get_text(' ', strip=True)
        # Ho passem a llista
        audiencias1 = audiencias1.split(" ")
        
        audiencias = pd.DataFrame(audiencias1, columns=['Audiencia'])
        cadenas = pd.DataFrame(cadenas1, columns=['Cadena'])

        df = pd.concat([cadenas, audiencias], axis=1, sort=False) 
        df["Fecha"] = day
        
        data = pd.concat([data, df]) 
    
    return data

In [3]:
new_month = 1
new_data = __audiencia_del_mes(new_month)

local_url = r'C:\Users\lula-\Documents\UOC Data Science\Tipologia\export_month_'+str(new_month)+'.csv'
new_data.to_csv (local_url, index = False, header=True)

print (new_data)

C:\Users\lula-\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



   Audiencia          Cadena       Fecha
0      14,9%             la1  2020-01-01
1      11,6%         antena3  2020-01-01
2      10,3%       telecinco  2020-01-01
3       5,8%          cuatro  2020-01-01
4       4,7%         lasexta  2020-01-01
..       ...             ...         ...
21      0,7%  disney-channel  2020-01-31
22      0,7%             ten  2020-01-31
23      0,5%         bemadtv  2020-01-31
24      0,3%  real-madrid-tv  2020-01-31
25      0,2%     teledeporte  2020-01-31

[806 rows x 3 columns]
